Import Libraries
TO DO
1) Add larger test dataset
2) Add Speed Tests
3) Create Real Polygons for Testing Purpose

In [1]:
import numpy as np
from osgeo import gdal
from scipy import stats
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor
import matplotlib.pyplot as plt

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

In [2]:
## Warning You will need 65 gb to create the memory map for the CU imper sum down data type

In [3]:
Grid_Name  = "data\\nlcd_2011_impervious_2011_edition_2014_10_10\\nlcd_2011_impervious_2011_edition_2014_10_10.img"
memory_map_Name = "data\\nlcd_imp.mmp"
Shapefile_Name = "test\\testPolys.shp"

In [21]:
r = gdal.Open(Grid_Name)
r.GetGeoTransform()
r.RasterXSize
band = r.GetRasterBand(1)


1 + float(band.ReadAsArray(10000,20000,1,1))
r.GetRasterBand(1).DataType


data = np.memmap(memory_map_Name, dtype='float32', mode='w+', shape=(r.RasterYSize,r.RasterXSize))
 
 

In [22]:
##Rewrite to do by columns
def dtFZS(ybnd,xbnd,out,band):
    for yw in range(0,ybnd):
        print (yw/ybnd)
        for xw in range(0,xbnd):
            out[yw,xw] = out[yw-1,xw] + float(band.ReadAsArray(xw,yw,1,1))

In [24]:
def rmp(Yt,Xt,Y,X,y,x):
    DY = (floor((Y-1)/y)) * y; dy = Y - DY;
    DX = (floor((X-1)/x)) * x; dx = X - DX;
 
    p1 = DX * Yt;
    p2 = DY * x;
    p3 = y * (dx-1) ;
    p4 = dy   
    pos = p1 + p2 + p3 + p4;
    return(int(pos-1));


In [112]:
# Function to read the raster as arrays for the chosen block size.
def create_sum_down(x_block_size, y_block_size,raster,mmp):
    ds = gdal.Open(raster)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    blocks = 0
    for y in range(0, ysize, y_block_size):
        print(y/ysize)
        if y + y_block_size < ysize:
            rows = y_block_size
        else:
            rows = ysize - y
        for x in range(0, xsize, x_block_size):
            if x + x_block_size < xsize:
                cols = x_block_size
            else:
                cols = xsize - x
            array = band.ReadAsArray(x, y, cols, rows)
            ##print(cols)
            #print(y)           
            mmp[y:y+rows,x:x+cols] = array  ## + mmp[y-1:y+rows-1,x:x+cols]
            del array
            blocks += 1
    band = None
    ds = None


In [117]:
data.shape[0]

104424

In [119]:
def part2(mmp):
    for yw in range(1,mmp.shape[0]):
        print(ysize,yw)
        for xw in range(mmp.shape[1]):
            mmp[yw,xw] += mmp[yw-1,xw]

In [ ]:
part2(data)

In [114]:
##read_raster(256,256,Grid_Name,data)

In [65]:
x = np.zeros(shape=(10,10))
y = np.zeros(shape=(100))
x[0:10,0:10] = y.reshape(10,10)

In [ ]:
##dtFZS(r.RasterYSize,r.RasterXSize,data,band)


In [6]:
class dtFZS:
    def __init__(self, img, memory_map_Name):

        self.r    = gdal.Open(img)
        self.band = self.r.GetRasterBand(1)
        ##self.g = np.array(self.r.GetRasterBand(1).ReadAsArray())
        
        self.clSze = self.r.GetGeoTransform()[1]
        self.ulX = self.r.GetGeoTransform()[0]
        self.ulY = self.r.GetGeoTransform()[3]
        
        self.RasterXSize = self.r.RasterXSize
        
        self.RasterYSize = self.r.RasterYSize
        
        self.brX = (self.r.RasterXSize * self.clSze) + self.ulX
        self.brY = self.ulY - (self.r.RasterYSize * self.clSze)   
        
        self.geo = self.r.GetGeoTransform()
        
        def dtFZS(ybnd,xbnd,out,band):
            for yw in range(0,ybnd+1):
                for xw in range(0,xbnd+1):
                    out[yw,xw] = out[yw-1,xw] + float(band.ReadAsArray(xw,yw,1,1))

        self.data = np.memmap(memory_map_Name, dtype='float32', mode='w+', shape=(self.RasterYSize,self.RasterXSize))
        
        dtFZS(self.RasterYSize,self.RasterXSize,self.data,self.band)
        


In [7]:
## Create Class for this grid
##dFZS = dtFZS(Grid_Name)
dFZS = dtFZS(Grid_Name,memory_map_Name)

TypeError: float() argument must be a string or a number, not 'NoneType'

Create Helper Functions to move between world and grid cordinates

In [36]:
## Input (world cords; x1, x2, Grid Upper Left, and Cellsize)
## Returns (grid cords; list(x))
## x1 and x2 represent vertexes on a polygon in order

def x_world_to_grid(wx1,wx2,dtFZS):
    
    x1a = min(wx1,wx2)
    x2a = max(wx1,wx2)
    
    xstart = ceil  ((x1a - dtFZS.ulX - (dtFZS.clSze/2)) / dtFZS.clSze)
    xstop =  floor ((x2a - dtFZS.ulX - (dtFZS.clSze/2)) / dtFZS.clSze)

    return(list(range(xstart,xstop+1)))

In [37]:
## Input (grid cords; list(x))
## Returns(world cords centroid ;(list(X)))

def x_grid_to_world(gx,dtFZS):
    return [i * dtFZS.clSze + dtFZS.ulX + (dtFZS.clSze/2) for i in gx]

In [38]:
## Input(world cords centroid ;(list(X)))
## Returns(world cords centroid ;(list(y)))

def y_world_pred(wx1,wx2,wy1,wy2,wxpred):
    slope = 0;
    if (wx1 > wx2):
        wx1,wx2,wy1,wy2 = wx2,wx1,wy2,wy1
    if(wx1 != wx2):    
        slope = (wy2-wy1)/(wx2-wx1)

    return [(slope * (i-wx1)) + wy1 for i in wxpred]

In [39]:
## Input(world cords centroid ;(list(y)))
## Returns (grid cords; list(y))

def y_world_to_grid(wypred,dtFZS):
        return [floor((dtFZS.ulY - i - (dtFZS.clSze/2)) / dtFZS.clSze ) for i in wypred] 

In [40]:
## Input(world Cords for 2 vertexes on a polygon, TL corner of grid, and cellsize)
## Returns(y,x grid cords)

def xy_grid(wx1,wx2,wy1,wy2,dtFZS):
    xgrid  = x_world_to_grid(wx1,wx2,dtFZS)
    wxpred = x_grid_to_world(xgrid,dtFZS)  
    wypred = y_world_pred(wx1,wx2,wy1,wy2,wxpred)
    ygrid  = y_world_to_grid(wypred,dtFZS)
    
    ## Remove negative arrays above
    idx = np.array([True if i >= 0 else False for i in ygrid],dtype=bool)
    
    xgrid = np.array(xgrid)[idx]
    ygrid = np.array(ygrid)[idx]
    
    cords = ((ygrid,xgrid))
    return cords

In [41]:
## Input(list of cords in world, summed down grid, real-world grid tl, cllSze)
## Output Sum v in polygon

def FstZS(geom,DS_FstZS):
    v = geom.exterior.coords  #have not tested
    total = 0;    
    for p in range(1,(len(v)+0)):
        if v[p][0] > v[p-1][0]:
            cords = xy_grid(v[p][0],v[p-1][0],v[p][1],v[p-1][1],DS_FstZS)
            total += np.sum(DS_FstZS.data[cords])
        elif v[p][0] < v[p-1][0]: 
            cords = xy_grid(v[p][0],v[p-1][0],v[p][1],v[p-1][1],DS_FstZS)
            total -= np.sum(DS_FstZS.data[cords])
    return int(total)

In [42]:
def test(geom):
    
    FstZSout = FstZS(geom, dFZS)
    Rastsout = zonal_stats(geom, Grid_Name,stats= "sum")
    
    return (float(FstZSout) , float(Rastsout[0]['sum']) )

In [43]:
## Input(albers conical equal area shapefile input of polygon test features)
## Output polygon geometry for each feature in input shapefile

driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(Shapefile_Name, 0)
layer = dataSource.GetLayer()

for feature in layer:
    geom = feature.GetGeometryRef()
    test(geom)

NameError: name 'dFZS' is not defined

In [44]:
#print(tst)
plt.scatter(y=[t[1] for t in tst],x= [t[0] for t in tst])
plt.xlabel("Fast Zonal Method")
plt.ylabel("Raster Stats")              
plt.show()   

NameError: name 'tst' is not defined